In [1]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl import app
from absl import flags

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
import tensorflow_lattice as tfl
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Import helpers
import import_ipynb
from common import buildDatasetForLattice
from common import scaleVolume
from common import extractLatticeWeights
from common import dropColumns
from common import filterBad
from common import splitDataset
from common import normDataset
from common import evaluatePerf
from common import evaluateCustom
from common import extractXGWeights

importing Jupyter notebook from common.ipynb


In [3]:
dataset, columns = buildDatasetForLattice()
dataset = filterBad(dataset, 54)
train_dataset, test_dataset, train_labels, test_labels = splitDataset(dataset, 0.2)
train_stats = train_dataset.describe().transpose()

Rejected 6196563 points (56.849770%)


In [4]:
preprocessed_dataset = dataset.copy()
col = preprocessed_dataset.first_val
preprocessed_dataset.first_val = np.log(col + 1)

In [5]:
kp_initializers = {
    'row': "quantile", 'rising_idx': "quantile", 'falling_idx': "quantile", 'first_val': "uniform", 'last_val': "uniform"
}
monotonicities = {
    "rising_idx": -1,
    "falling_idx": -1
}

In [6]:
keras.backend.clear_session()

In [12]:
def buildLattice(num_keypoints, lattice_size, columns):
    inputs = [keras.layers.Input(shape=[1]) for _ in columns]
    combined_calibrators = []
    for inpt, ft in zip(inputs, columns):
        if ft != "row":
            if kp_initializers[ft] == "quantile":
                quantile_vals = [i/(num_keypoints - 1.0) for i in range(num_keypoints)]
                keypoints = dataset[ft].quantile(quantile_vals).values
            if kp_initializers[ft] == "uniform":
                keypoints = np.linspace(preprocessed_dataset[ft].min(), preprocessed_dataset[ft].max(), num=num_keypoints)

            calibrator = tfl.layers.PWLCalibration(
                input_keypoints=keypoints, dtype=tf.float32, output_min=0.0, output_max=lattice_size - 1.0,
                kernel_regularizer = [("wrinkle", 1e-4, 1e-5)],
                monotonicity=monotonicities.get(ft, 0),
            )(inpt)
        else:
            # row is categorical
            calibrator = tfl.layers.CategoricalCalibration(
                num_buckets=preprocessed_dataset[ft].nunique(),
                output_min = 0.0,
                output_max = lattice_size - 1.0,
            )(inpt)
        combined_calibrators.append(calibrator)
    lattice = tfl.layers.Lattice(
        lattice_sizes=[lattice_size for _ in columns],
        monotonicities=['increasing' if (ft == 'rising_idx' or ft == 'falling_idx') else 'none' for x in columns],
        output_min=dataset['delay'].min(),
        output_max=dataset['delay'].max())(keras.layers.concatenate(combined_calibrators))
    
    model = keras.models.Model(inputs=inputs, outputs=lattice)
    model.compile(loss=keras.losses.mean_absolute_error,
                optimizer=keras.optimizers.Adam(), metrics=["mse"])
    return model


In [17]:
def trainLattice(model, cols, epochs):
    features = [preprocessed_dataset[col].values for col in cols]
    target = preprocessed_dataset["delay"]

    model.fit(features,
            target,
            batch_size=32,
            epochs=epochs,
            validation_split=0.2,
            shuffle=False, workers=32, use_multiprocessing=True, callbacks=[
                keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto', restore_best_weights=True)])
    return model

In [20]:
show_perf = True
train_model = True
load_model=False

for num_kps, lattice_size in [(4, 2), (8, 2), (16, 2), (4, 3), (4, 4), (4, 5), (8, 3), (8, 4)][5:]:
    model_path = f"./hypercube_sweep_{num_kps}_{lattice_size}.h5"
    print("Beginning:", num_kps, lattice_size)

    if load_model:
        model = keras.models.load_model(model_path, custom_objects={
            "CategoricalCalibration": tfl.layers.CategoricalCalibration,
            "PWLCalibration": tfl.layers.PWLCalibration,
            "Lattice": tfl.layers.Lattice,
            "Linear": tfl.layers.Linear
        })
    else:
        model = buildLattice(num_kps, lattice_size, columns[:-1])
    history = trainLattice(model, columns[:-1], 40)
    model.save(model_path)

Beginning: 4 5
Train on 3762662 samples, validate on 940666 samples
Epoch 1/40
3762662/3762662 [==============================] - 384s 102us/sample - loss: 374.7888 - mse: 275606.8750 - val_loss: 374.3428 - val_mse: 256742.0312
Epoch 2/40
3762662/3762662 [==============================] - 380s 101us/sample - loss: 347.2587 - mse: 232136.2812 - val_loss: 318.3040 - val_mse: 205969.5781
Epoch 3/40
3762662/3762662 [==============================] - 381s 101us/sample - loss: 291.9383 - mse: 181858.6875 - val_loss: 267.1119 - val_mse: 164587.1562
Epoch 4/40
3762662/3762662 [==============================] - 378s 100us/sample - loss: 245.7229 - mse: 147546.0469 - val_loss: 227.4076 - val_mse: 138077.5312
Epoch 5/40
3762662/3762662 [==============================] - 380s 101us/sample - loss: 212.0789 - mse: 127144.4062 - val_loss: 199.6695 - val_mse: 121214.0156
Epoch 6/40
3762662/3762662 [==============================] - 380s 101us/sample - loss: 189.0234 - mse: 112286.2109 - val_loss: 181.

3762662/3762662 [==============================] - 303s 81us/sample - loss: 74.8091 - mse: 26246.2539 - val_loss: 66.2958 - val_mse: 22674.0195
Epoch 13/40
3762662/3762662 [==============================] - 310s 82us/sample - loss: 61.6754 - mse: 20256.5723 - val_loss: 59.3089 - val_mse: 19132.7734
Epoch 14/40
3762662/3762662 [==============================] - 313s 83us/sample - loss: 55.9959 - mse: 17277.8477 - val_loss: 54.3523 - val_mse: 16304.6982
Epoch 15/40
3762662/3762662 [==============================] - 313s 83us/sample - loss: 51.7485 - mse: 14144.4219 - val_loss: 49.9132 - val_mse: 13022.0225
Epoch 16/40
3762662/3762662 [==============================] - 312s 83us/sample - loss: 47.0988 - mse: 11100.9385 - val_loss: 45.4832 - val_mse: 10176.3584
Epoch 17/40
3762662/3762662 [==============================] - 308s 82us/sample - loss: 42.1368 - mse: 8287.3750 - val_loss: 40.0774 - val_mse: 7153.0288
Epoch 18/40
3762662/3762662 [==============================] - 308s 82us/sampl

In [16]:
model.save(model_path)